# Data Cleaning Pipeline - DTP Project

## Overview
Notebook ini mendokumentasikan proses pembersihan data lowongan kerja (sekitar 12 ribu baris) dengan kombinasi aturan deterministik dan imputasi menggunakan Gemini API.

## Tujuan
- Menyusun ulang dan melengkapi data lowongan kerja.
- Mengurangi missing values melalui pemetaan referensi dan AI.
- Menyiapkan dataset untuk SFT (Supervised Fine-Tuning) dan DPO (Direct Preference Optimization).

## Dataset
- **Input utama**: `dataset.xlsx`
- **Data referensi**: `PON TIK FIX - Sheet1.csv`
- **Hasil akhir**: `cleaned_data_final.xlsx`

## Prasyarat
- Lingkungan Google Colab.
- File dataset dan environment `gemini.env` tersedia di Google Drive.
- Akses API Gemini yang masih berlaku.

---

**Ringkasan Perubahan**
- Memperbarui pipeline pembersihan data lowongan dengan dokumentasi langkah demi langkah di notebook data_clean.ipynb.
- Membersihkan dan menormalkan dataset 9001-12057, termasuk imputasi level pekerjaan, industri, status, skillset, tools, dan deskripsi menggunakan Gemini API.
- Mengekspor hasil akhir ke cleaned_data_final.xlsx dan mengganti dataset sumber menjadi `data_raw/Data Lowongan Pekerjaan 9001-12057 - Sheet1.xlsx`.

**Catatan Pengujian**
- Notebook dieksekusi penuh di Google Colab; seluruh sel kode selesai tanpa error.
- Hasil akhir diperiksa melalui ringkasan missing values dan sampel data.

## Langkah 1 – Mount Google Drive
Gunakan perintah berikut untuk memasang Google Drive sehingga notebook dapat mengakses file dataset, file referensi, dan environment `gemini.env`.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Langkah 2 – Inisialisasi Library dan Konfigurasi Gemini
Impor dependensi utama (pandas, numpy, dotenv, dan SDK Gemini), muat environment `gemini.env`, lalu konfigurasi model `gemini-2.5-flash-lite` agar siap digunakan untuk proses imputasi.

In [2]:
import pandas as pd
import numpy as np
import google.generativeai as genai
import os
from dotenv import load_dotenv

# load dot env
env_file_name = "/content/drive/MyDrive/olah_data_dtp/gemini.env"

#fetch api gemini
if load_dotenv(dotenv_path=env_file_name):
    print(f"File{env_file_name} berhasil dimuat..")
else:
    print(f"File {env_file_name} tidak berhasil dimuat..")

api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    raise ValueError(f"API KEY gagal dimuat pada file {env_file_name}..")


genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-flash-lite')
print(f"konfig berhasil")

File/content/drive/MyDrive/olah_data_dtp/gemini.env berhasil dimuat..
konfig berhasil


## Langkah 3 – Memuat Dataset dan Menormalkan Nilai Kosong
Membaca data referensi PON TIK serta dataset lowongan kerja dari Google Drive, kemudian mengganti string tertentu seperti "Tidak Ada" atau tanda strip menjadi `NaN` untuk memudahkan imputasi lanjutan.

In [27]:
try:
    df_pon_tik =  pd.read_csv("/content/drive/MyDrive/olah_data_dtp/PON TIK FIX - Sheet1.csv", encoding='latin-1')
    df_lowongan = pd.read_excel("/content/drive/MyDrive/olah_data_dtp/dataset.xlsx")
except FileNotFoundError as e:
    print(f"Error: {e}. Pastikan file CSV berada di direktori yang sama.")
    exit()

missing_values = ["Tidak Ada", "Tidak ada", "-", "", "Not Applicable"]

df_lowongan.replace(missing_values, np.nan, inplace=True)
print("Jumlah missing values awal di beberapa kolom:")
print(df_lowongan[['Level Pekerjaan', 'Industri', 'Spesial Info', 'Skillset', 'Tools']].isnull().sum())

Jumlah missing values awal di beberapa kolom:
Level Pekerjaan    1997
Industri            720
Spesial Info         15
Skillset              2
Tools               343
dtype: int64


## Langkah 4 – Imputasi Awal Level Pekerjaan
Membuat kamus pemetaan okupasi ke level berdasarkan data PON TIK, kemudian menggunakan fungsi `impute_level_pekerjaan` untuk mengisi kolom `Level Pekerjaan` yang kosong di dataset utama.

In [5]:
okupasi_to_level = df_pon_tik.set_index('OKUPASI')['LEVEL'].to_dict()

def impute_level_pekerjaan(row):
    if pd.notna(row['Level Pekerjaan']):
        return row['Level Pekerjaan']

    if pd.isna(row['Okupasi']):
        return None

    # clean okupasi row
    okupasi_lowongan = str(row['Okupasi'])

    okupasi_upper = okupasi_lowongan.upper()

    pos_separate = okupasi_upper.find(' - LEVEL')

    if pos_separate != -1:
        okupasi = okupasi_upper[:pos_separate].strip()
    else:
        okupasi = okupasi_upper.strip()

    for okupasi_pon_tik, level in okupasi_to_level.items():
        if okupasi in str(okupasi_pon_tik).upper():
            return level


    return None

df_lowongan['Level Pekerjaan'] = df_lowongan.apply(impute_level_pekerjaan, axis=1)

print("\nImputasi selesai")


Imputasi selesai


## Langkah 5 – Kategorisasi Level Pekerjaan
Mengonversi level numerik menjadi label kategori yang lebih mudah dibaca (misalnya Internship, Associate, hingga Direktur). Hasil kategorisasi ini juga ditampilkan untuk peninjauan cepat.

In [6]:
#pemetaan
def kategorisasi_level(level):
    try:
        level = int(float(level))
        if level <= 2:
            return 'Internship/Magang/OJT'
        elif level <= 4:
            return 'Lulusan Baru/Junior/Entry Level/Fresh Graduate'
        elif level == 5:
            return 'Associate'
        elif level == 6:
            return 'Mid Senior Level'
        elif level == 7:
            return 'Supervisor/Asisten Manager'
        elif level >= 8:
            return 'Direktur/Eksekutif'
        else:
            return np.nan
    except (ValueError, TypeError):
        return level

# categorized
df_lowongan['Level Pekerjaan'] = df_lowongan['Level Pekerjaan'].apply(kategorisasi_level)

print("Kategorisasi 'Level Pekerjaan' selesai.")
print("\nContoh hasil setelah perbaikan:")
print(df_lowongan[['Okupasi', 'Level Pekerjaan']].head())

Kategorisasi 'Level Pekerjaan' selesai.

Contoh hasil setelah perbaikan:
                                   Okupasi  \
0            ICT PROJECT MANAGER - Level 7   
1  ASSISTANT BACK-END PROGRAMMER - Level 4   
2       CLOUD COMPUTING ENGINEER - Level 7   
3              BACKEND DEVELOPER - Level 6   
4                   DATA ANALYST - Level 6   

                                  Level Pekerjaan  
0                                Mid-Senior level  
1                           Internship/Magang/OJT  
2                                Mid-Senior level  
3                                Mid-Senior level  
4  Lulusan Baru/Junior/Entry Level/Fresh Graduate  


## Langkah 6 – Pemeriksaan Sampel Data
Melihat 100 baris pertama dari dataframe untuk memastikan hasil imputasi awal dan kategorisasi sudah sesuai ekspektasi.

In [19]:
df_lowongan.head(100)

,No,Pekerjaan,Level Pekerjaan,Status Pekerjaan,Industri,Spesial Info,Deskripsi Pekerjaan,Jenis_Okupasi,Okupasi,Area Fungsi,Skillset,Tools
0,9000,Sr. Project Manager - Data Center,Mid-Senior level,Full-time,Real Estate,Project Management and Information Technology,Job Title Sr. Project Manager - Data Center Jo...,PON-TIK,ICT PROJECT MANAGER - Level 7,Tata Kelola Teknologi Informasi,project management;delivery;project oversight;...,data center
1,9001,Back End Developer Intern,Internship/Magang/OJT,Internship,Software Development,NaN,We are looking for a qualified candidate to jo...,PON-TIK,ASSISTANT BACK-END PROGRAMMER - Level 4,Pengembangan Produk Digital,api design;api development;authentication;auth...,express.js;flask;gin;git;golang;jwt;mongodb;mysql
2,9002,Senior DevOps Engineer,Mid-Senior level,Full-time,Financial Services and IT Services and IT Cons...,Information Technology,JOB OVERVIEW We are looking for a highly skill...,PON-TIK,CLOUD COMPUTING ENGINEER - Level 7,Teknologi Dan Infrastruktur,devops;infrastructure engineering;cloud platfo...,terraform;ansible;cloudformation;aws;gcp;alicl...
3,9003,Back End Engineer (Supply Chain) - Sea Labs,Mid-Senior level,Full-time,"Software Development, Internet Marketplace Pla...",Information Technology,About The Team: About Sea Labs Sea Labs Indone...,PON-TIK,BACKEND DEVELOPER - Level 6,Pengembangan Produk Digital,system design;implementation;software engineer...,golang;python;c++;java;http;tcp;mysql;redis
4,9004,xxCompanySecretxx - Data Analyst,Lulusan Baru/Junior/Entry Level/Fresh Graduate,Full-time,Software Development,Information Technology,Data Analyst About The Role Strap on your helm...,PON-TIK,DATA ANALYST - Level 6,Sains Data-Kecerdasan Artifisial,"analisis data, business acumen, forecasting, p...","SQL, Tableau, Looker, Metabase, Power BI, Goog..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,9095,Gen AI Fullstack Engineer,Associate,Contract,"IT Services and IT Consulting and Technology, ...",Information Technology,Job Description Build GenAI pipelines using LL...,PON-TIK,LEAD DATA ENGINEER - Level 8,Sains Data-Kecerdasan Artifisial,"fullstack development, GenAI pipelines, RESTfu...","Python, Flask/FastAPI, OpenAI, Gemini, Claude,..."
96,9096,.NET Developer,Lulusan Baru/Junior/Entry Level/Fresh Graduate,Full-time,IT Services and IT Consulting,Engineering and Information Technology,Company Description PT. Kairos Utama Indonesia...,PON-TIK,WEB DEVELOPER - Level 6,Pengembangan Produk Digital,object-oriented programming;programming;softwa...,.net;.net core;asp.net mvc
97,9097,Backend Golang Developer,Mid-Senior level,Contract,IT Services and IT Consulting,Engineering and Information Technology,Qualification: 3+ Year’s Experience in softwar...,PON-TIK,BACKEND DEVELOPER - Level 6,Pengembangan Produk Digital,software development;agile methodology;api dev...,golang;java spring boot;sql server;mysql
98,9098,Java Developer,Lulusan Baru/Junior/Entry Level/Fresh Graduate,Full-time,IT Services and IT Consulting,Engineering and Information Technology,Job Description Understand and implement requi...,PON-TIK,ASSOCIATE PROGRAMMER - Level 5,Pengembangan Produk Digital,requirement analysis;implementation;developmen...,java;springboot;graalvm;postgre;mysql;sql serv...


## Langkah 7 – Definisi Prompt Gemini untuk Industri, Spesial Info, Skillset, dan Tools
Mendefinisikan fungsi `impute_with_gemini_final` yang akan memanggil Gemini API guna mengisi kolom-kolom kompetensi dan metadata perusahaan ketika data asli kosong.

In [8]:
import time
import numpy as np
import pandas as pd

def impute_with_gemini_final(row, column_to_impute):
    if pd.notna(row[column_to_impute]):
        return row[column_to_impute]

    pekerjaan = row['Pekerjaan']
    deskripsi = row['Deskripsi Pekerjaan']

    if pd.isna(pekerjaan) or pd.isna(deskripsi):
        return np.nan

    # buat prompt
    prompts = {
        'Industri': f"Berdasarkan pekerjaan '{pekerjaan}' dan deskripsi '{deskripsi}', apa nama industri yang paling sesuai? Berikan satu jawaban singkat saja. Contoh: Teknologi Informasi.",
        'Spesial Info': f"Dari deskripsi pekerjaan '{deskripsi}' untuk posisi '{pekerjaan}', identifikasi 1-2 kualifikasi khusus yang paling menonjol. Jika tidak ada, tulis 'Tidak ada'. Berikan jawaban singkat.",
        'Skillset': f"Berdasarkan deskripsi '{deskripsi}' untuk posisi '{pekerjaan}', sebutkan 5 skill utama yang dibutuhkan, pisahkan dengan titik koma.",
        'Tools': f"Berdasarkan deskripsi '{deskripsi}' untuk posisi '{pekerjaan}', sebutkan 3 tools/software utama yang digunakan, pisahkan dengan titik koma."
    }

    prompt = prompts.get(column_to_impute)
    if not prompt:
        return np.nan

    try:
        response = model.generate_content(prompt)


        time.sleep(4)

        return response.text.strip()
    except Exception as e:
        print(f"Error pada baris {row.name} untuk kolom '{column_to_impute}': {e}")
        return np.nan

## Langkah 8 – Imputasi Batch untuk Kolom Industri, Spesial Info, Skillset, dan Tools
Menjalankan fungsi Gemini secara baris per baris untuk mengisi empat kolom penting. Proses ini membutuhkan jeda (`time.sleep`) agar tetap dalam batasan rate limit API.

In [9]:
columns_to_impute_gemini = ['Industri', 'Spesial Info', 'Skillset', 'Tools']

for col in columns_to_impute_gemini:
    print(f"\nMemulai imputasi penuh untuk kolom: {col} (ini akan memakan waktu)...")
    df_lowongan[col] = df_lowongan.apply(
        lambda row: impute_with_gemini_final(row, col) if pd.isna(row[col]) else row[col],
        axis=1
    )
    print(f"Imputasi untuk kolom '{col}' selesai.")

print("\nProses imputasi menggunakan Gemini telah selesai sepenuhnya.")


Memulai imputasi penuh untuk kolom: Industri (ini akan memakan waktu)...
Imputasi untuk kolom 'Industri' selesai.

Memulai imputasi penuh untuk kolom: Spesial Info (ini akan memakan waktu)...
Imputasi untuk kolom 'Spesial Info' selesai.

Memulai imputasi penuh untuk kolom: Skillset (ini akan memakan waktu)...
Imputasi untuk kolom 'Skillset' selesai.

Memulai imputasi penuh untuk kolom: Tools (ini akan memakan waktu)...
Imputasi untuk kolom 'Tools' selesai.

Proses imputasi menggunakan Gemini telah selesai sepenuhnya.


## Langkah 9 – Ringkasan Missing Values Setelah Imputasi Pertama
Memeriksa ulang jumlah nilai kosong di setiap kolom untuk menilai efektivitas tahap imputasi pertama.

In [40]:
df_lowongan.isna().sum()

,0
No,0
Pekerjaan,0
Level Pekerjaan,33
Status Pekerjaan,0
Industri,2
Spesial Info,0
Deskripsi Pekerjaan,2
Jenis_Okupasi,0
Okupasi,0
Area Fungsi,0


## Langkah 10 – Menyimpan Hasil Sementara
Mengekspor dataframe yang sudah diperkaya ke file Excel di Google Drive. File ini menjadi checkpoint sebelum dilakukan imputasi lanjutan.

In [11]:
df_lowongan.to_excel("/content/drive/MyDrive/olah_data_dtp/cleaned_data.xlsx", index=False)

## Langkah 11 – Memuat Ulang Data untuk Imputasi Tahap Berikutnya
Membaca kembali file Excel yang sudah diperkaya dan menghitung ulang missing values sebagai baseline sebelum melakukan imputasi lanjutan.

In [12]:
df_cleaned = pd.read_excel("/content/drive/MyDrive/olah_data_dtp/cleaned_data.xlsx")
df_cleaned.isna().sum()

,0
No,0
Pekerjaan,0
Level Pekerjaan,33
Status Pekerjaan,0
Industri,2
Spesial Info,0
Deskripsi Pekerjaan,2
Jenis_Okupasi,0
Okupasi,0
Area Fungsi,0


## Langkah 12 – Definisi Prompt Gemini untuk Level dan Status Pekerjaan
Mendefinisikan ulang fungsi imputasi dengan prompt berbeda untuk mengisi `Level Pekerjaan`, `Status Pekerjaan`, dan `Tools` pada dataframe hasil tahap sebelumnya.

In [13]:
import time
import numpy as np
import pandas as pd

def impute_with_gemini_final(row, column_to_impute):
    if pd.notna(row[column_to_impute]):
        return row[column_to_impute]

    pekerjaan = row['Pekerjaan']
    deskripsi = row['Deskripsi Pekerjaan']
    level = row['Level Pekerjaan']

    if pd.isna(pekerjaan) or pd.isna(deskripsi):
        return np.nan

    # buat prompt
    prompts = {
        'Level Pekerjaan': f"Berdasarkan pekerjaan '{pekerjaan}' dan deskripsi '{deskripsi}', apa level pekerjaan yang paling sesuai? Jawaban singkat terdiri dari Associate, Direktur/Eksekutif, Internship/Magang/OJT, Lulusan Baru/Junior/Entry Level/Fresh Graduate, Mid Senior Level, Supervisor/Asisten Manager.",
        'Status Pekerjaan': f"Dari deskripsi pekerjaan '{deskripsi}' untuk posisi '{pekerjaan}', dan level pekerjaan '{level}', apa status pekerjaan yang paling sesuai? Jawaban singkat terdiri dari Full Time, Kontrak/Temporer, Internship, Kasual, Part-Time.",
        'Tools': f"Berdasarkan deskripsi '{deskripsi}' untuk posisi '{pekerjaan}', sebutkan 3 tools/software utama yang digunakan, pisahkan dengan titik koma."
    }

    prompt = prompts.get(column_to_impute)
    if not prompt:
        return np.nan

    try:
        response = model.generate_content(prompt)


        time.sleep(4)

        return response.text.strip()
    except Exception as e:
        print(f"Error pada baris {row.name} untuk kolom '{column_to_impute}': {e}")
        return np.nan

## Langkah 13 – Imputasi Batch untuk Level dan Status Pekerjaan
Menjalankan fungsi Gemini untuk tiga kolom tambahan. Tahap ini memperhalus informasi senioritas dan status hubungan kerja pada setiap lowongan.

In [14]:
columns_to_impute_gemini = ['Level Pekerjaan', 'Status Pekerjaan', 'Tools']

for col in columns_to_impute_gemini:
    print(f"\nMemulai imputasi penuh untuk kolom: {col} (ini akan memakan waktu)...")
    df_cleaned[col] = df_cleaned.apply(
        lambda row: impute_with_gemini_final(row, col) if pd.isna(row[col]) else row[col],
        axis=1
    )
    print(f"Imputasi untuk kolom '{col}' selesai.")

print("\nProses imputasi menggunakan Gemini telah selesai sepenuhnya.")


Memulai imputasi penuh untuk kolom: Level Pekerjaan (ini akan memakan waktu)...
Imputasi untuk kolom 'Level Pekerjaan' selesai.

Memulai imputasi penuh untuk kolom: Status Pekerjaan (ini akan memakan waktu)...
Imputasi untuk kolom 'Status Pekerjaan' selesai.

Memulai imputasi penuh untuk kolom: Tools (ini akan memakan waktu)...
Imputasi untuk kolom 'Tools' selesai.

Proses imputasi menggunakan Gemini telah selesai sepenuhnya.


## Langkah 14 – Evaluasi Missing Values Setelah Tahap Kedua
Mengukur kembali jumlah nilai kosong untuk memastikan proses imputasi tahap kedua telah menutupi sebagian besar kekosongan yang tersisa.

In [15]:
df_cleaned.isna().sum()

,0
No,0
Pekerjaan,0
Level Pekerjaan,0
Status Pekerjaan,0
Industri,2
Spesial Info,0
Deskripsi Pekerjaan,2
Jenis_Okupasi,0
Okupasi,0
Area Fungsi,0


## Langkah 15 – Inspeksi Baris yang Masih Memiliki Nilai Kosong
Menampilkan baris yang masih memiliki nilai `NaN` untuk dianalisis lebih lanjut sebelum menjalankan imputasi tahap akhir.

In [21]:
display(df_cleaned[df_cleaned.isnull().any(axis=1)])

,No,Pekerjaan,Level Pekerjaan,Status Pekerjaan,Industri,Spesial Info,Deskripsi Pekerjaan,Jenis_Okupasi,Okupasi,Area Fungsi,Skillset,Tools
1764,10764,IT Network & Support,Internship/Magang/OJT,Full time,NaN,Administrasi Jaringan & Sistem (Teknologi Info...,NaN,PON-TIK,NETWORK SUPPORT TECHNICIAN STAFF - Level 2,Teknologi Dan Infrastruktur,network management;technical support;troublesh...,cisco ios;windows server;linux;active director...
1874,10874,IT Support,Mid Senior Level,Full time,NaN,Administrasi Jaringan & Sistem (Teknologi Info...,NaN,PON-TIK,IT SUPPORT SUPERVISOR - Level 6,Pengembangan Produk Digital,technical support;troubleshooting;problem solv...,windows;linux;macos;microsoft office;teamviewe...


## Langkah 16 – Definisi Prompt Gemini untuk Deskripsi dan Industri
Menyesuaikan prompt Gemini terakhir agar dapat merekonstruksi `Deskripsi Pekerjaan` dan `Industri` berdasarkan data yang sudah dilengkapi pada tahap sebelumnya.

In [22]:
import time
import numpy as np
import pandas as pd

def impute_with_gemini_final(row, column_to_impute):
    if pd.notna(row[column_to_impute]):
        return row[column_to_impute]

    pekerjaan = row['Pekerjaan']
    level = row['Level Pekerjaan']
    spesial_info = row['Spesial Info']
    skillset = row['Skillset']
    tools = row['Tools']

    # buat prompt
    prompts = {
        'Deskripsi Pekerjaan': f"Berdasarkan pekerjaan '{pekerjaan}' pada level '{level}',  spesial info '{spesial_info}', skillset '{skillset}', dan tools '{tools}', buatkan deskripsi singkat lowongan pekerjaan untuk posisi/role ini. Jawaban singkat terdiri dari deskripsi lowongan pekerjaan.",
        'Industri': f"Berdasarkan pekerjaan '{pekerjaan}' pada level '{level}',  spesial info '{spesial_info}', skillset '{skillset}', dan tools '{tools}', apa nama industri yang paling sesuai? Berikan satu jawaban singkat saja. Contoh: Teknologi Informasi.",
        }

    prompt = prompts.get(column_to_impute)
    if not prompt:
        return np.nan

    try:
        response = model.generate_content(prompt)


        time.sleep(4)

        return response.text.strip()
    except Exception as e:
        print(f"Error pada baris {row.name} untuk kolom '{column_to_impute}': {e}")
        return np.nan

## Langkah 17 – Imputasi Batch untuk Deskripsi dan Industri
Menjalankan Gemini untuk menghasilkan narasi deskripsi pekerjaan dan menentukan industri yang paling relevan, dengan memanfaatkan informasi level, skillset, dan tools yang sudah lengkap.

In [23]:
columns_to_impute_gemini = ['Deskripsi Pekerjaan', 'Industri']

for col in columns_to_impute_gemini:
    print(f"\nMemulai imputasi penuh untuk kolom: {col} (ini akan memakan waktu)...")
    df_cleaned[col] = df_cleaned.apply(
        lambda row: impute_with_gemini_final(row, col) if pd.isna(row[col]) else row[col],
        axis=1
    )
    print(f"Imputasi untuk kolom '{col}' selesai.")

print("\nProses imputasi menggunakan Gemini telah selesai sepenuhnya.")


Memulai imputasi penuh untuk kolom: Deskripsi Pekerjaan (ini akan memakan waktu)...
Imputasi untuk kolom 'Deskripsi Pekerjaan' selesai.

Memulai imputasi penuh untuk kolom: Industri (ini akan memakan waktu)...
Imputasi untuk kolom 'Industri' selesai.

Proses imputasi menggunakan Gemini telah selesai sepenuhnya.


## Langkah 18 – Validasi Akhir Missing Values
Memastikan seluruh kolom sudah terisi setelah tiga tahap imputasi. Jika masih ada nilai kosong, tahap remediasi tambahan perlu dipertimbangkan.

In [24]:
df_cleaned.isna().sum()

,0
No,0
Pekerjaan,0
Level Pekerjaan,0
Status Pekerjaan,0
Industri,0
Spesial Info,0
Deskripsi Pekerjaan,0
Jenis_Okupasi,0
Okupasi,0
Area Fungsi,0


## Langkah 19 – Menyimpan Hasil Akhir
Mengekspor dataframe final yang telah dibersihkan dan dilengkapi ke `cleaned_data_final.xlsx` sebagai output yang siap digunakan untuk pipeline downstream.

In [28]:
df_cleaned.to_excel("/content/drive/MyDrive/olah_data_dtp/cleaned_data_final.xlsx")